In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import iris
import tobac
import sys
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
from cartopy import crs as ccrs
from cartopy import util as cutil
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter

%matplotlib inline

In [ ]:
mode = "Moisture Mode"

diri = "/scratch/gb02/mr4682/data/regridded/UM/not_removing_tc/"

if mode == "Moisture Mode":
    mode_str = "moisture.mode"
    mode_var_name = "moisture_mode"
    v_max = 9.0
elif mode == "Mixed System":
    mode_str = "mixed.system"
    mode_var_name = "mixed_system"
    v_max = 27.0
elif mode == "IG Wave":
    mode_str = "ig.wave"
    mode_var_name = "ig_wave"
    v_max = 50.0
elif mode == "Tropical Cyclone":
    modes_str = "tc"
    mode_var_name = mode_str
    v_max = 10.0
else:
    sys.exit(f"There is no mode named {mode}!")

filei = f"olr.zoom10.to.0p25deg.{mode_str}.nc"

In [ ]:
ds = xr.open_dataset(f"{diri}{filei}")

In [ ]:
time_start = "2020-12-01 00:00:00"
time_end = "2021-02-28 21:00:00"

In [ ]:
time = ds["time"].sel(time=slice(time_start, time_end))
latitude = ds["latitude"]
longitude = ds["longitude"]

In [ ]:
time

In [ ]:
olr = ds[f"rlut_{mode_var_name}"].sel(time=slice(time_start, time_end)).compute()
olr

In [ ]:
olr_stddev = olr.std(ddof=1)

In [ ]:
olr_standardised = olr / olr_stddev
olr_standardised = olr_standardised.fillna(0.0)

In [ ]:
dxy = np.radians(longitude.values[1] - longitude.values[0]) * 6.371e+06
dt = (time.values[1] - time.values[0]) / np.timedelta64(1, "s")

In [ ]:
parameters_features = {}
parameters_features["threshold"] = [-2.0, -2.5, -3.0, -3.5, -4.0]
parameters_features["target"] = "minimum"
parameters_features["position_threshold"] = "weighted_diff"
parameters_features["sigma_threshold"] = 1.0
parameters_features["n_min_threshold"] = 4
parameters_features["PBC_flag"] = "hdim_2"

In [ ]:
Features = tobac.feature_detection_multithreshold(olr_standardised, dxy, **parameters_features)

In [ ]:
Features

In [ ]:
parameters_linking = {}
parameters_linking["v_max"] = v_max
parameters_linking["stubs"] = 3
parameters_linking["method_linking"] = "predict"
parameters_linking["PBC_flag"] = "hdim_2"
parameters_linking["min_h2"] = 0
parameters_linking["max_h2"] = len(longitude) - 1

In [ ]:
Track = tobac.linking_trackpy(Features, olr_standardised, dt=dt, dxy=dxy, **parameters_linking)

In [ ]:
parameters_segmentation = {}
parameters_segmentation["target"] = "minimum"
parameters_segmentation["method"] = "watershed"
parameters_segmentation["threshold"] = -2.0
parameters_segmentation["PBC_flag"] = "hdim_2"

In [ ]:
Mask, Track = tobac.segmentation_2D(Track, olr_standardised, dxy, **parameters_segmentation)

In [ ]:
time_plot_start = "2021-01-01 00:00:00"
time_plot_end = "2021-01-07 21:00:00"

time_plot = time.sel(time=slice(time_plot_start, time_plot_end))

In [ ]:
figx = 12.0
figy = 8.0

fig = plt.figure(figsize=(figx, figy))
plt.close()

def update(frame):
    fig.clf()
    time_frame = np.datetime_as_string(time_plot.values[frame], unit="s")
    olr_plot = olr.sel(time=time_frame)
    
    Features_plot = Features.loc[pd.to_datetime(Features["timestr"]) == pd.to_datetime(time_frame)]
    Mask_plot = Mask.sel(time=time_frame)
    
    Mask_plot, lon_plot = cutil.add_cyclic(Mask_plot, x=Mask_plot["longitude"])
    
    ax = fig.add_axes([1.0 / figx, 2.0 / figy, 10.0 / figx, 5.0 / figy], projection=ccrs.PlateCarree(central_longitude=210.0), aspect="auto")
    plot = ax.pcolormesh(olr_plot["longitude"], olr_plot["latitude"], olr_plot, transform=ccrs.PlateCarree(), shading="nearest", cmap="RdBu_r", vmin=-60.0, vmax=60.0, edgecolors="face")
    plot_boundary = ax.contour(lon_plot, olr_plot["latitude"], Mask_plot, transform=ccrs.PlateCarree(), levels=[0.5], colors="black")
    plot_marker = ax.scatter(x=Features_plot["longitude"], y=Features_plot["latitude"], s=15, c="yellow", marker="x", transform=ccrs.PlateCarree())
    
    ax.coastlines()
    ax.set_title(f"{mode}", loc="left", fontsize=18.0)
    ax.set_title(time_frame, loc="right", fontsize=18.0)
    ax.set_xticks(np.arange(0.0, 360.0, 60.0), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(-30.0, 45.0, 15.0), crs=ccrs.PlateCarree())
    ax.tick_params(labelsize=18.0)
    ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
    ax.yaxis.set_major_formatter(LatitudeFormatter())
    
    cb_ax = fig.add_axes([0.25, 0.75 / figy, 0.5, 0.5 * figx / 20 / figy])
    
    cb = fig.colorbar(plot, cax=cb_ax, orientation="horizontal", extend="both")
    
    cb_ax.set_xlabel("OLR Anomaly [W m**-2]", fontsize=18.0)
    cb_ax.tick_params(labelsize=18.0)

animation = ani.FuncAnimation(fig=fig, func=update, frames=len(time_plot), interval=750)

HTML(animation.to_html5_video())